In [1]:
from open_clip import create_model_from_pretrained, get_tokenizer

model, preprocess = create_model_from_pretrained('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')
tokenizer = get_tokenizer('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')

/home/armand/miniconda3/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/armand/miniconda3/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/armand/miniconda3/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


open_clip_pytorch_model.bin:   0%|          | 0.00/784M [00:00<?, ?B/s]

open_clip_config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

/home/armand/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

/home/armand/miniconda3/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/armand/miniconda3/lib/python3.10/site-packages/open_clip/factory.py:129: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_gl

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/225k [00:00<?, ?B/s]

In [29]:
import torch
from urllib.request import urlopen
from PIL import Image

dataset_url = 'https://huggingface.co/microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224/resolve/main/example_data/biomed_image_classification_example_data/'
test_imgs = [
    'squamous_cell_carcinoma_histopathology.jpeg',
    'H_and_E_histopathology.jpg',
    'bone_X-ray.jpg',
    'adenocarcinoma_histopathology.jpg',
    'covid_line_chart.png',
    'IHC_histopathology.jpg',
    'chest_X-ray.jpg',
    'brain_MRI.jpg',
    'pie_chart.png'
]

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.eval()


context_length = 256

images = torch.stack([preprocess(Image.open(urlopen(dataset_url + img))) for img in test_imgs]).to(device)
texts = tokenizer(["" for im in images], context_length=context_length).to(device)
with torch.no_grad():
    image_features, text_features, logit_scale = model(images, texts)

print(image_features.shape)

torch.Size([9, 512])


In [11]:
with torch.no_grad():
    image_features, text_features, logit_scale = model(images, texts)
print(image_features[:,:3])

tensor([[-0.0279,  0.0556, -0.2120],
        [ 0.0046, -0.0682, -0.2127],
        [ 0.0295, -0.0657, -0.2510],
        [-0.0197,  0.0414, -0.1855],
        [ 0.0547,  0.0121, -0.0997],
        [ 0.0119, -0.0210, -0.2200],
        [-0.0434, -0.0100, -0.2650],
        [ 0.0150, -0.0132, -0.2424],
        [-0.0262, -0.0522, -0.2326]], device='cuda:0')


In [ ]:
from fff.subject_model import BiomedClipModel

In [84]:
compiled_model = BiomedClipModel(model, tokenizer)

In [85]:
im_feat = compiled_model.encode(to_tensor(Image.open(urlopen(dataset_url + "H_and_E_histopathology.jpg"))).unsqueeze(0).to(device))

In [86]:
images = torch.stack([preprocess(Image.open(urlopen(dataset_url + "H_and_E_histopathology.jpg")))]).to(device)
texts = tokenizer([""], context_length=0).to(device)

im_feat_orig, _, _ = model(images, texts)

In [87]:
torch.mean((im_feat_orig - im_feat)**2)

tensor(1.5150e-08, device='cuda:0', grad_fn=<MeanBackward0>)

In [88]:
torch.save(compiled_model, "SubjectModels/saved_models/BiomedClip.pt")

In [79]:
print(im_feat.shape)

torch.Size([1, 512])


In [81]:
%ls

 BiomedModel.ipynb                     'FIF PythAE benchmark'/
'FFF Boltzmann Generator Evaluation'/  'Sine Experiments.ipynb'
 Fiber_Learning/                        SubjectModels/
 FiberModels/


In [3]:
import torch

compiled_model = torch.load("SubjectModels/saved_models/BiomedClip.pt", weights_only=False)

In [4]:
compiled_model

BiomedClipModel(
  (model): CustomTextCLIP(
    (visual): TimmModel(
      (trunk): VisionTransformer(
        (patch_embed): PatchEmbed(
          (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
          (norm): Identity()
        )
        (pos_drop): Dropout(p=0.0, inplace=False)
        (patch_drop): Identity()
        (norm_pre): Identity()
        (blocks): Sequential(
          (0): Block(
            (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
            (attn): Attention(
              (qkv): Linear(in_features=768, out_features=2304, bias=True)
              (q_norm): Identity()
              (k_norm): Identity()
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=768, out_features=768, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
            )
            (ls1): Identity()
            (drop_path1): Identity()
            (norm2): LayerNorm((768,), eps=1e-06, elementwise_af

In [35]:
import torch.nn.functional as F


class BiomedClipModel(torch.nn.Module):
    def __init__(self, model, tokenizer, image_only=True):
        super().__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.image_only = image_only

    def encode(self, x, c=None):
        if x.ndim == 2:
            x = x.reshape(x.shape[0], *guess_image_shape(prod(x.shape[1:])))
        if self.image_only:
            text = self.tokenizer(["" for i in range(len(x))], context_length=0).to(
                x.device
            )
        else:
            text = self.tokenizer(c, context_length=256).to(x.device)

        images = self.preprocess(x)
        image_features, text_features, logit_scale = self.model(images, text)
        image_features_alt = F.normalize(self.model.visual(images))
        print(f"Difference = {torch.mean((image_features - image_features_alt)**2)}")
        
        if self.image_only:
            return image_features
        return image_features, text_features

    def decode(self, x, c=None):
        raise (RuntimeError("BiomedClip has no decoder"))

    def preprocess(self, img):
        _, ch, h, w = img.shape

        if h < w:
            new_h, new_w = 224, int(w * 224 / h)  # Scale width
        else:
            new_w, new_h = 224, int(h * 224 / w)  # Scale height

        img = F.interpolate(
            img,
            size=(new_h, new_w),
            mode="bicubic",
            align_corners=False,
            antialias=True,
        )

        # Center crop manually
        top = (new_h - 224) // 2
        left = (new_w - 224) // 2

        img = img[..., top : top + 224, left : left + 224]

        # Convert to RGB (if needed)
        if ch == 1:  # Grayscale input
            img = img.repeat(1, 3, 1, 1)  # Expand to RGB channels
        img = (img + 1)/2
        print(img.min(), img.max())
        # Normalize
        mean = torch.tensor(
            [0.48145466, 0.4578275, 0.40821073], device=img.device
        ).view(1, 3, 1, 1)
        std = torch.tensor(
            [0.26862954, 0.26130258, 0.27577711], device=img.device
        ).view(1, 3, 1, 1)
        img = (img - mean) / std

        return img

In [36]:
new_model = BiomedClipModel(compiled_model.model, compiled_model.tokenizer, image_only=True)

In [37]:
from fff.data import load_dataset

data_set_config = {
    "name": "ldct",
    "root": "/home/hd/hd_hd/hd_gu452/ldct_data",
    "condition": "lowdose",
    "data": "highdose",
    "patchsize": 512,
    "resize_to": 224,
    "augment": False,
    "data_norm": "clipped",
}
_, val_ds, _ = load_dataset(**data_set_config)
test_image = val_ds[0][0].unsqueeze(0).to("cuda").reshape(1, 1, 224, 224)

In [38]:
new_model.encode(test_image)

tensor(0., device='cuda:0') tensor(0.8645, device='cuda:0')
Difference = 0.0


tensor([[ 2.3878e-02, -1.2688e-02, -2.6265e-01, -4.4693e-02, -7.4737e-04,
         -8.2396e-02,  1.7068e-02,  1.4796e-03,  8.3227e-03, -2.0871e-02,
          4.0072e-02,  3.7047e-03, -2.9414e-02, -3.7666e-02,  6.6381e-02,
         -8.9713e-02, -2.5352e-02,  2.4048e-02,  9.7350e-04,  3.1916e-02,
          4.9677e-03, -1.9399e-02,  6.4550e-03,  6.9542e-03,  3.1073e-02,
          2.5827e-02, -2.0630e-02, -5.2746e-03,  3.9451e-02,  1.2690e-02,
          2.5855e-02,  6.2013e-02,  3.3095e-02, -2.9158e-02,  1.8586e-02,
          1.5496e-02,  7.2155e-02,  4.7854e-02, -7.7180e-02, -5.9647e-02,
         -1.6639e-02,  2.3054e-02, -7.6917e-03,  5.4932e-02,  6.0708e-02,
         -2.1040e-02, -5.0919e-02, -5.0424e-02,  1.3263e-02, -4.2999e-02,
         -5.6058e-02, -8.6828e-03, -5.2775e-03, -4.3339e-02, -1.4808e-02,
         -4.8698e-03, -9.4112e-03,  1.4438e-02,  2.8399e-02,  2.3740e-02,
         -8.4587e-03, -2.3908e-02, -5.1646e-02,  6.1793e-02,  1.9655e-02,
         -4.4434e-02, -3.3231e-02, -7.

In [43]:
samples = []
for i, dp in enumerate(val_ds):
    if i >= 100:
        break
    samples.append(dp[0])

samples = torch.cat(samples, dim=0)
samples.mean(), samples.std()

(tensor(-0.6465), tensor(0.3470))